<a href="https://colab.research.google.com/github/remidion/PGM-Project/blob/main/Timeliness.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/remidion/PGM-Project/blob/main/main.ipynb)

In [1]:
# Imports
from getpass import getpass
import pandas as pd
import os
import numpy as np
import glob

In [2]:
!pip install geopandas
import geopandas as gpd

     |████████████████████████████████| 972kB 6.4MB/s 
     |████████████████████████████████| 14.8MB 305kB/s 
     |████████████████████████████████| 6.5MB 16.5MB/s 


# Data Preparation

## Download MBTA Data

In [3]:
if not os.path.exists("data"):
  !wget https://www.arcgis.com/sharing/rest/content/items/d685ba39d9a54d908f49a2a762a9eb47/data
if not os.path.exists("MBTA Bus Arrival Departure Aug-Sept 2018.csv"):
  !unzip data

if not os.path.exists("data.1"):
  !wget https://www.arcgis.com/sharing/rest/content/items/1bd340b39942438685d8dcdfe3f26d1a/data
if not os.path.exists("MBTA Bus Arrival Departure Apr-June 2019.csv"):
  !unzip data.1

--2020-12-06 02:43:32--  https://www.arcgis.com/sharing/rest/content/items/d685ba39d9a54d908f49a2a762a9eb47/data
Resolving www.arcgis.com (www.arcgis.com)... 52.23.2.231, 34.199.206.244, 34.233.149.104, ...
Connecting to www.arcgis.com (www.arcgis.com)|52.23.2.231|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://ago-item-storage.s3.us-east-1.amazonaws.com/d685ba39d9a54d908f49a2a762a9eb47/MBTA_Bus_Arrival_Departure_Times_2018.zip?X-Amz-Security-Token=IQoJb3JpZ2luX2VjEHIaCXVzLWVhc3QtMSJHMEUCIBK%2B%2BcNRe1oLQPG6AhBzNvyjvEkjj6wSf9Ku3vhhgwfSAiEAmQuQKMVensTnTWX%2Fml4sFNasI6FecyAISb6H8PA23g8qvQMI%2Bv%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FARAAGgw2MDQ3NTgxMDI2NjUiDGYIj8t5hUx9zbTCfyqRA0A%2FcQw4UW4fWaJ1%2Fb%2B%2B7S0CIILCCTNMb9NvS97ouhPimmfi4uE0Kzt3qhPWBtYHDw6a035%2Fbbm8IusbfNdbbUiOJLgfX53Aam3%2F1YeNUCTTY7g6z%2B6ZZhSlZr2OASLBrW22k%2FFOrsRHoYHnyK%2Bu54DZm8STjEZvjfn1hU0%2FMQTDAviYci6KkOmlZINiTLLm3Orv8w3R2heUN5J3G2nR0QcRuEckVlByAx7Xc0DfoIN8bs3IK%2FR151Z%2FIGjkSfbMAf2vff9A

## Load MBTA Arrival Data

In [4]:
df2 = pd.read_csv("MBTA Bus Arrival Departure Apr-June 2019.csv")
df2

,service_date,route_id,direction,half_trip_id,stop_id,stop_name,stop_sequence,point_type,standard_type,scheduled,actual,scheduled_headway,headway
0,2019-04-01,01,Inbound,42976988.0,75,mit,4.0,Midpoint,Schedule,1900-01-01 05:19:00,1900-01-01 05:21:20,NaN,NaN
1,2019-04-01,01,Inbound,42976988.0,79,hynes,5.0,Midpoint,Schedule,1900-01-01 05:23:00,1900-01-01 05:24:17,NaN,NaN
2,2019-04-01,01,Inbound,42976988.0,187,masta,6.0,Midpoint,Schedule,1900-01-01 05:25:00,1900-01-01 05:26:05,NaN,NaN
3,2019-04-01,01,Inbound,42976988.0,59,Wasma,7.0,Midpoint,Schedule,1900-01-01 05:29:00,1900-01-01 05:27:59,NaN,NaN
4,2019-04-01,01,Inbound,42977170.0,110,hhgat,1.0,Startpoint,Headway,1900-01-01 05:30:00,1900-01-01 05:29:55,1200.0,1230.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7670643,2019-06-30,SL5,Outbound,44045095.0,15176,mawor,8.0,Midpoint,Headway,1900-01-02 00:54:00,NaN,900.0,NaN
7670644,2019-06-30,SL5,Outbound,44045095.0,55,Wasma,9.0,Midpoint,Headway,1900-01-02 00:55:00,NaN,900.0,NaN
7670645,2019-06-30,SL5,Outbound,44045095.0,60,Walen,10.0,Midpoint,Headway,1900-01-02 00:56:00,NaN,900.0,NaN
7670646,2019-06-30,SL5,Outbound,44045095.0,61,Melwa,11.0,Midpoint,Headway,1900-01-02 00:57:00,NaN,900.0,NaN


## Utility functions

In [5]:
def filter_on_row_col(df, target_rows, target_columns):
  for key, value in target_rows:
      df.drop(df[df[key] != value].index, inplace=True)
  df.drop(df.columns.difference(target_columns), axis=1, inplace=True)

def load_targets_from_csvs(target_columns, target_rows):
  bus_arrival_files = glob.glob('./*.csv')

  df_cumulative = pd.DataFrame()
  for f in bus_arrival_files:
    df_tmp = pd.read_csv(f)
    filter_on_row_col(df_tmp, target_rows, target_columns)
    df_cumulative = df_cumulative.append(df_tmp)
    print(f"Loaded {f}")
    del df_tmp

  return df_cumulative

# EDA on Routes Timeliness

In [6]:
df2 = load_targets_from_csvs(target_columns=['route_id','direction','stop_id'], target_rows=[])

Loaded ./MBTA Bus Arrival Departure Oct-Dec 2018.csv
Loaded ./MBTA Bus Arrival Departure Jan-Mar 2019.csv
Loaded ./MBTA Bus Arrival Departure Oct-Dec 2019.csv
Loaded ./MBTA Bus Arrival Departure Jul-Sept 2019.csv
Loaded ./MBTA Bus Arrival Departure Aug-Sept 2018.csv
Loaded ./MBTA Bus Arrival Departure Apr-June 2019.csv


In [7]:
# Calculate visits per stop
group = df2[['route_id','direction','stop_id']].groupby(['route_id','direction'])
unique_stops = group.nunique().rename(columns={"stop_id":"unique_stop_ids"})
total_stops = pd.DataFrame(group.size()).rename(columns={0:"total_stops"})
stops = pd.merge(unique_stops, total_stops, left_index=True, right_index=True)
stops['visits_per_stop'] = stops['total_stops'] / stops['unique_stop_ids']

In [8]:
stops.sort_values('visits_per_stop', ascending=False).head(20)

unique_stop_ids  total_stops  visits_per_stop
route_id direction                                               
SL5      Outbound                12       814194     67849.500000
         Inbound                 13       872944     67149.538462
111      Inbound                 12       777438     64786.500000
SL1      Outbound                 7       448292     64041.714286
31       Outbound                 5       304750     60950.000000
         Inbound                  5       300727     60145.400000
SL2      Outbound                 6       345890     57648.333333
         Inbound                  6       335191     55865.166667
111      Outbound                13       718718     55286.000000
23       Inbound                 10       549635     54963.500000
SL1      Inbound                 13       709980     54613.846154
01       Outbound                 9       488393     54265.888889
         Inbound                  9       472691     52521.222222
39       Inbound                 11       561718     51065.272727
28       Inbound                 11       552235     50203.181818
39       Outbound                10       496903     49690.300000
743      Outbound                11       544599     49509.000000
77       Outbound                11       538967     48997.000000
         Inbound                 11       536599     48781.727273
23       Outbound                12       548555     45712.916667

# Preprocess data

Loading the data for a single route and a single direction

In [9]:
target_columns = ["service_date", "half_trip_id", "stop_id", "stop_sequence", "scheduled", "actual"]
target_rows = [('route_id', '01'),('direction', 'Inbound')]
df3 = load_targets_from_csvs(target_columns, target_rows)

Loaded ./MBTA Bus Arrival Departure Oct-Dec 2018.csv
Loaded ./MBTA Bus Arrival Departure Jan-Mar 2019.csv
Loaded ./MBTA Bus Arrival Departure Oct-Dec 2019.csv
Loaded ./MBTA Bus Arrival Departure Jul-Sept 2019.csv
Loaded ./MBTA Bus Arrival Departure Aug-Sept 2018.csv
Loaded ./MBTA Bus Arrival Departure Apr-June 2019.csv


Comment:
stop_id and stop_sequence are not always consistent in the dataframe!

In [10]:
group = df3.groupby(by=['stop_id', 'stop_sequence']).size()
print(group)

stop_id  stop_sequence
59       5.0                  1
         6.0                  3
         7.0              42083
62       1.0                  2
         6.0                  1
         7.0                  3
         8.0              42080
64       1.0                180
         2.0                  2
         7.0                  1
         8.0                  3
         9.0              42079
67       1.0                  3
         2.0              42069
72       1.0                  1
         2.0                  3
         3.0              42069
75       2.0                  1
         3.0                  3
         4.0              42083
79       3.0                  1
         4.0                  3
         5.0              42083
110      1.0              42070
187      4.0                  1
         5.0                  3
         6.0              42083
dtype: int64


Create a definition of valid stop IDs, based on the frequency of occurence

In [11]:
valid_stop_ids = group[group > group.max()/2]
sequence_to_id = valid_stop_ids.index.to_frame().reset_index(drop=True).sort_values(['stop_sequence']).set_index(['stop_id'])
print(sequence_to_id)
sequence_to_id = sequence_to_id['stop_sequence'].to_dict()

         stop_sequence
stop_id               
110                1.0
67                 2.0
72                 3.0
75                 4.0
79                 5.0
187                6.0
59                 7.0
62                 8.0
64                 9.0


Drop invalid (out of sequence) stop IDs

In [12]:
df3['expected_stop_sequence'] = df3['stop_id'].apply(lambda x: sequence_to_id[x])
df3.drop(df3[((df3['expected_stop_sequence'] != df3['stop_sequence']) | (df3['stop_sequence'].isna()))].index, axis=0, inplace=True)
df3.drop('expected_stop_sequence', axis=1, inplace=True)

Drop incomplete routes

In [13]:
groups = df3.groupby(by=['half_trip_id']).size()
stop_sequence_length = int(max(sequence_to_id.values()))
valid_half_trip_ids = pd.Series(groups[groups == stop_sequence_length].index).rename('valid_half_trip_ids')
df3 = df3.loc[df3['half_trip_id'].isin(valid_half_trip_ids)]

Pivot data frame

In [14]:
df4 = df3.pivot(index=['service_date', 'half_trip_id'], columns=['stop_sequence'], values=['scheduled',	'actual'])
df4

scheduled  ...               actual
stop_sequence                              1.0  ...                  9.0
service_date half_trip_id                       ...                     
2018-08-02   40135101.0    1900-01-01 05:10:00  ...  1900-01-01 05:37:40
             40135103.0    1900-01-01 06:21:00  ...  1900-01-01 06:50:37
             40135105.0    1900-01-01 07:44:00  ...  1900-01-01 08:37:11
             40135121.0    1900-01-01 07:06:00  ...  1900-01-01 07:43:01
             40135155.0    1900-01-01 06:06:00  ...  1900-01-01 06:41:32
...                                        ...  ...                  ...
2019-09-30   45106585.0    1900-01-01 18:33:00  ...  1900-01-01 19:09:43
             45106587.0    1900-01-01 20:11:00  ...  1900-01-01 20:45:59
             45106589.0    1900-01-01 21:33:00  ...  1900-01-01 22:00:24
             45108415.0    1900-01-01 08:49:00  ...                  NaN
             45108654.0    1900-01-01 18:18:00  ...  1900-01-01 19:01:43

[40646 rows x 18 columns]

# Training

## TODO

# Inference

## TODO